# 06 — Sensitivity to Unobserved Confounding

We compute sequential sensitivity bounds for OPE under a Gamma-bounded
confounding model. The bounds widen as Gamma increases.

## Setup

```
pip install "causalrl[plots]"
```

In [1]:
from __future__ import annotations

import numpy as np

from crl.benchmarks.mdp_synth import SyntheticMDP, SyntheticMDPConfig
from crl.sensitivity.namkoong2020 import GammaSensitivityModel, confounded_ope_bounds
from crl.utils.seeding import set_seed
from crl.viz.plots import plot_sensitivity_curve

In [2]:
set_seed(0)
np.random.seed(0)

## Compute sensitivity bounds

In [3]:
benchmark = SyntheticMDP(SyntheticMDPConfig(seed=0, horizon=5))
dataset = benchmark.sample(num_trajectories=200, seed=1)

gammas = np.array([1.0, 1.25, 1.5, 2.0])
curve = confounded_ope_bounds(dataset, benchmark.target_policy, gammas)
curve.to_dict()

{'gammas': array([1.  , 1.25, 1.5 , 2.  ]),
 'lower': array([-3.08284048, -4.07269858, -5.03009032, -6.89617424]),
 'upper': array([-3.08284048, -2.24712439, -1.64939738, -0.81092695]),
 'metadata': {'method': 'namkoong2020_sequential'}}

In [ ]:
records = [
    {"gamma": g, "lower": lo, "upper": up}
    for g, lo, up in zip(curve.gammas, curve.lower, curve.upper)
]
fig = plot_sensitivity_curve(records)
fig

In [ ]:
print(
    "Gamma=1.0 bounds: "
    f"[{curve.lower[0]:.3f}, {curve.upper[0]:.3f}]"
)

## Gamma sensitivity model internals

The GammaSensitivityModel exposes the multiplicative adjustments applied to
trajectory returns under bounded confounding.

In [4]:
returns = np.sum(
    dataset.rewards * (dataset.discount ** np.arange(dataset.horizon)),
    axis=1,
)
gamma_model = GammaSensitivityModel(gamma=1.5)
low_adj, high_adj = gamma_model.adjustments(returns)
low_adj[:5], high_adj[:5]

(array([0.66666667, 0.66666667, 0.66666667, 0.66666667, 1.5       ]),
 array([1.5       , 1.5       , 1.5       , 1.5       , 0.66666667]))

## Takeaways

- Gamma controls the strength of unobserved confounding.
- Larger Gamma yields wider (more conservative) bounds.
- Sensitivity analysis complements standard OPE estimates.